<a href="https://colab.research.google.com/github/NathanVuSwinburne/Structural-Defect-Detection-AI-for-Structural-Engineering/blob/remove-dockerise/COS40007_finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from google.colab import drive

if 'google.colab' in sys.modules:
  drive.mount('/content/drive')
  os.chdir('/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Check if we are using GPU or not...
!nvidia-smi

Sat May 24 05:02:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project


In [ ]:
!pip install ultralytics

In [ ]:
from IPython import display
from IPython.display import display, Image, clear_output
clear_output()
import ultralytics
ultralytics.checks()
!pip install roboflow

Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 42.4/235.7 GB disk)


In [ ]:


from roboflow import Roboflow
rf = Roboflow(api_key="sGMMjyN0L8bBltD06Jzu")
project = rf.workspace("cos40007-workspace").project("cos40007-final-project")
version = project.version(8)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to COS40007-Final-Project-8 in yolov8:: 100%|██████████| 6100/6100 [00:35<00:00, 170.27it/s]


In [ ]:
# YOLO Dataset Processor for Object Detection and Segmentation
# This notebook splits a multi-class YOLO dataset into separate datasets for:
# - Bounding box detection (Classes 0-1: Crack, Rust)
# - Segmentation (Class 2: Tower Structure)

# Install required packages
!pip install opencv-python
!pip install pyyaml

import os
import shutil
import glob
import yaml
from pathlib import Path
import numpy as np
import cv2
from ultralytics import YOLO
import argparse
from google.colab import drive
import matplotlib.pyplot as plt


# Define paths - Modify these depending on your dataset location
DATASET_ROOT = "/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-8"  # Default Colab working directory

# Create paths for split datasets
BBOX_DATASET = os.path.join(DATASET_ROOT, "bbox_dataset")
SEG_DATASET = os.path.join(DATASET_ROOT, "seg_dataset")

# Function to create directory structure
def create_directory_structure():
    """Create the directory structure for both datasets"""
    for dataset_path in [BBOX_DATASET, SEG_DATASET]:
        for split in ["train", "valid", "test"]:
            for subdir in ["images", "labels"]:
                os.makedirs(os.path.join(dataset_path, split, subdir), exist_ok=True)
    print("✅ Directory structure created")

# Function to split a label file into bbox and segmentation components
def split_label_file(input_file, bbox_output_file, seg_output_file):
    """Split a YOLO label file into separate bbox and segmentation files"""
    bbox_lines = []
    seg_lines = []

    with open(input_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue

            class_id = int(parts[0])

            # Class 0 (Crack) and Class 1 (Rust) are bounding boxes
            if class_id == 0 or class_id == 1:
                bbox_lines.append(line)
            # Class 2 (Tower Structure) is segmentation
            elif class_id == 2:
                # Adjust class_id to 0 for the segmentation dataset
                new_line = "0" + line[1:]
                seg_lines.append(new_line)

    # Only write files if they have content
    if bbox_lines:
        with open(bbox_output_file, 'w') as f:
            f.writelines(bbox_lines)

    if seg_lines:
        with open(seg_output_file, 'w') as f:
            f.writelines(seg_lines)

    return bool(bbox_lines), bool(seg_lines)

# Function to process a dataset split
def process_split(split):
    """Process a dataset split (train/valid/test)"""
    labels_dir = os.path.join(DATASET_ROOT, split, "labels")
    images_dir = os.path.join(DATASET_ROOT, split, "images")

    bbox_labels_dir = os.path.join(BBOX_DATASET, split, "labels")
    bbox_images_dir = os.path.join(BBOX_DATASET, split, "images")

    seg_labels_dir = os.path.join(SEG_DATASET, split, "labels")
    seg_images_dir = os.path.join(SEG_DATASET, split, "images")

    print(f"Processing {split} split...")
    print(f"Source images: {images_dir}")

    # Check if source directories exist
    if not os.path.exists(labels_dir):
        print(f"⚠️ Warning: Source labels directory {labels_dir} does not exist")
        return

    if not os.path.exists(images_dir):
        print(f"⚠️ Warning: Source images directory {images_dir} does not exist")
        return

    # Process each label file in the split
    processed_bbox = 0
    processed_seg = 0
    label_files = glob.glob(os.path.join(labels_dir, "*.txt"))

    for i, label_file in enumerate(label_files):
        if i % 100 == 0:
            print(f"Processing file {i+1}/{len(label_files)}...")

        base_name = os.path.basename(label_file)
        image_name = os.path.splitext(base_name)[0] + ".jpg"  # Assuming .jpg extension

        # Define output paths
        bbox_output_file = os.path.join(bbox_labels_dir, base_name)
        seg_output_file = os.path.join(seg_labels_dir, base_name)

        # Split the label file
        has_bbox, has_seg = split_label_file(label_file, bbox_output_file, seg_output_file)

        # If the image exists, copy it to the appropriate dataset(s)
        image_path = os.path.join(images_dir, image_name)
        if os.path.exists(image_path):
            if has_bbox:
                shutil.copy(image_path, os.path.join(bbox_images_dir, image_name))
                processed_bbox += 1
            if has_seg:
                shutil.copy(image_path, os.path.join(seg_images_dir, image_name))
                processed_seg += 1

    print(f"✅ Processed {processed_bbox} images for bbox detection and {processed_seg} images for segmentation")

# Function to create YAML configuration files
def create_yaml_file(dataset_path, nc, names):
    """Create a YAML configuration file for the dataset"""
    yaml_content = {
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'nc': nc,
        'names': names
    }

    yaml_path = os.path.join(dataset_path, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    print(f"✅ Created YAML configuration at: {yaml_path}")

# Function to run inference and combine results
def run_inference_and_combine(bbox_model_path, seg_model_path, test_images_dir, output_dir):
    """Run inference with both models and combine results"""
    # Load models
    bbox_model = YOLO(bbox_model_path)
    seg_model = YOLO(seg_model_path)

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Get list of test images
    image_paths = glob.glob(os.path.join(test_images_dir, "*.jpg"))

    if not image_paths:
        print(f"⚠️ No images found in {test_images_dir}")
        return

    print(f"Running inference on {len(image_paths)} images...")

    # Run inference on all test images
    for i, image_path in enumerate(image_paths):
        if i % 10 == 0:
            print(f"Processing image {i+1}/{len(image_paths)}...")

        image_name = os.path.basename(image_path)

        # Load image
        img = cv2.imread(image_path)
        if img is None:
            print(f"⚠️ Failed to load image: {image_path}")
            continue

        # Run inference with both models
        bbox_results = bbox_model.predict(img, save=False, verbose=False)
        seg_results = seg_model.predict(img, save=False, verbose=False)

        # Create a new image with combined results
        result_img = img.copy()

        # Process bounding box detections (Crack and Rust)
        if bbox_results and len(bbox_results[0].boxes) > 0:
            boxes = bbox_results[0].boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                conf = float(box.conf[0])

                if conf > 0.25:  # Confidence threshold
                    # Class 0 is Crack (red color)
                    if cls == 0:
                        color = (0, 0, 255)  # BGR format: red
                        label = f"Crack {conf:.2f}"
                    # Class 1 is Rust (blue color)
                    else:
                        color = (255, 0, 0)  # BGR format: blue
                        label = f"Rust {conf:.2f}"

                    # Draw bounding box
                    cv2.rectangle(result_img, (x1, y1), (x2, y2), color, 2)

                    # Draw label
                    cv2.putText(result_img, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Process segmentation detections (Tower Structure)
        if seg_results and hasattr(seg_results[0], 'masks') and seg_results[0].masks is not None:
            masks = seg_results[0].masks

            # Create an overlay for the segmentation
            overlay = result_img.copy()

            for i, mask_tensor in enumerate(masks.data):
                # Convert mask tensor to numpy array and resize to image dimensions
                mask = mask_tensor.cpu().numpy()
                mask = cv2.resize(mask, (result_img.shape[1], result_img.shape[0]))

                # Apply the mask
                overlay[mask > 0.5] = overlay[mask > 0.5] * 0.7 + np.array([0, 255, 0]) * 0.3

            # Add the overlay to the result image
            cv2.addWeighted(overlay, 0.7, result_img, 0.3, 0, result_img)

            # Add label for Tower Structure
            cv2.putText(result_img, "Tower Structure", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Save the result image
        output_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_path, result_img)

        # Display a sample result every 10 images
        if i % 10 == 0:
            plt.figure(figsize=(12, 8))
            # Convert BGR to RGB for correct display
            plt_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            plt.imshow(plt_img)
            plt.title(f"Detection Results: {image_name}")
            plt.axis('off')
            plt.show()

    print(f"✅ Inference completed. Results saved to: {output_dir}")

# Main function to prepare datasets
def prepare_datasets():
    """Main function to prepare the datasets"""
    # Create directory structure
    create_directory_structure()

    # Process each split
    for split in ["train", "valid", "test"]:
        process_split(split)

    # Create YAML files for both datasets
    create_yaml_file(BBOX_DATASET, 2, ["Crack", "Rust"])
    create_yaml_file(SEG_DATASET, 1, ["Tower Structure"])

    print("\n✅ Dataset preparation completed!")
    print(f"📁 Bounding box dataset for YOLOv8m created at: {BBOX_DATASET}")
    print(f"📁 Segmentation dataset for YOLOv8m-seg created at: {SEG_DATASET}")

# Run inference from trained models
def run_inference():
    """Run inference using trained models"""
    bbox_model_path = input("Enter path to trained bbox model (e.g., runs/detect/train/weights/best.pt): ")
    seg_model_path = input("Enter path to trained segmentation model (e.g., runs/segment/train/weights/best.pt): ")
    test_images_dir = input(f"Enter path to test images directory (default: {DATASET_ROOT}/test/images): ") or f"{DATASET_ROOT}/test/images"
    output_dir = input(f"Enter path for output directory (default: {DATASET_ROOT}/combined_results): ") or f"{DATASET_ROOT}/combined_results"

    if not os.path.exists(bbox_model_path):
        print(f"❌ Bbox model not found at {bbox_model_path}")
        return

    if not os.path.exists(seg_model_path):
        print(f"❌ Segmentation model not found at {seg_model_path}")
        return

    run_inference_and_combine(bbox_model_path, seg_model_path, test_images_dir, output_dir)

# Interactive menu for Colab
def interactive_menu():
    """Display an interactive menu for the Colab notebook"""
    from IPython.display import clear_output

    while True:
        print("\n" + "="*50)
        print("🔍 YOLO Dataset Processor for Detection and Segmentation")
        print("="*50)

        print("1. Prepare Datasets (Split into bbox and segmentation)")
        print("2. Run Inference")
        print("3. Exit")
        print("="*50)

        choice = input("Enter your choice (1-5): ")
        clear_output(wait=True)

        if choice == '1':
            prepare_datasets()
        elif choice == '2':
            run_inference()
        elif choice == '3':
            print("Exiting...")
            break
        else:
            print("❌ Invalid choice. Please try again.")

if __name__ == "__main__":
    interactive_menu()

Exiting...


In [ ]:
%cd {HOME}

!yolo task=detect mode=train \
  model=yolov8m.pt \
  data={BBOX_DATASET}/data.yaml \
  epochs=100 imgsz=1280 plots=True \
  patience=5


/content
Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-8/bbox_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=Fa

In [ ]:
%cd {HOME}

# For detection model (cracks and rust)
!yolo task=detect mode=predict model=/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/runs/detect/train/weights/best.pt source=/content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-7/test/images conf=0.25 save=True

/content
Ultralytics 8.3.143 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,840,918 parameters, 0 gradients, 78.7 GFLOPs

image 1/44 /content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-7/test/images/BA_Knights_HIll_FM_Mast_00009_jpg.rf.d802a6d89a68dc8dcede1b4153cd6068.jpg: 1280x1280 1 Rust, 11.7ms
image 2/44 /content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-7/test/images/BA_Knights_HIll_FM_Mast_00013_jpg.rf.01f74c95027c13bdc79ffc7abb552377.jpg: 1280x1280 5 Rusts, 11.7ms
image 3/44 /content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-7/test/images/BA_Knights_HIll_FM_Mast_00023_jpg.rf.3f7e9939df5c66836cdace84c4b94209.jpg: 1280x1280 3 Rusts, 11.7ms
image 4/44 /content/drive/MyDrive/ColabNotebooks/COS40007/Final-Project/COS40007-Final-Project-7/test/images/BA_Knights_HIll_FM_Mast_00060_jpg.rf.ae51e3ae5905f610a1970e5378073652.jpg: